In [ ]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)
data_path = "/content/drive/MyDrive/Colab Notebooks/DL/"  # this is your drive

Mounted at /content/drive/


### 统计其他诊断编码的最小和最大长度

In [ ]:
import json

# 读取 JSON 文件
def load_json_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# 统计“其他诊断编码”字段中以分号分隔的文本数量
def count_diagnosis_segments(json_data, field_name="其他诊断编码"):
    segment_counts = []

    # 遍历 JSON 数据
    for item in json_data:
        if field_name in item and item[field_name]:  # 确保字段存在且不为空
            # 以分号分隔文本，并统计段数（空字符串也算一段）
            segments = item[field_name].split(';')
            # 过滤掉空字符串后统计有效段数
            valid_segments = [seg for seg in segments if seg.strip()]
            segment_counts.append(len(valid_segments))
        else:
            # 如果字段不存在或为空，记为 0
            segment_counts.append(0)

    return segment_counts

# 获取最小值和最大值
def get_min_max(counts):
    if not counts:  # 如果列表为空
        return None, None
    return min(counts), max(counts)

# 主函数
def main(file_path):
    # 加载 JSON 文件
    json_data = load_json_file(file_path)

    # 统计每条记录中“其他诊断编码”字段的分段数量
    segment_counts = count_diagnosis_segments(json_data)

    # 获取最小值和最大值
    min_count, max_count = get_min_max(segment_counts)

    # 输出结果
    if min_count is not None and max_count is not None:
        print(f"最小文本段数: {min_count}")
        print(f"最大文本段数: {max_count}")
    else:
        print("没有找到有效数据进行统计。")

# 示例用法
if __name__ == "__main__":
    # 假设 JSON 文件路径为 'data.json'
    file_path = data_path + 'train_data/ICD-Coding-train.json'
    main(file_path)

最小文本段数: 1
最大文本段数: 7


### 切分训练集和验证集

In [ ]:
import random
from sklearn.model_selection import train_test_split

# 读取你的JSON文件路径
json_file = data_path + 'train_data/extracted_text.json'  # 修改为你自己的JSON文件路径

with open(json_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

print(f"数据总量：{len(data)} 条")

def extract_text(diagnosis_list):
    """
    将单个诊断中的所有字段拼接为文本
    """
    text = []
    for diag in diagnosis_list:
        for key in ['疾病', '症状', '检查项目', '治疗方法', '身体部位', '药物', '手术']:
            value = diag.get(key, '')
            # 字符串格式直接拼接
            if isinstance(value, str):
                text.append(f"{key}：{value}")
            # 列表或其他格式，转为字符串
            elif isinstance(value, list):
                text.append(f"{key}：{'，'.join(value)}")
    return '；'.join(text)

dataset = []

for item in data:
    # 文本1和文本2
    text1_raw = item.get('抽取后主要诊断病例文本', [])
    text2_raw = item.get('抽取后其他诊断病例文本', [])

    text1 = extract_text(text1_raw)
    text2 = extract_text(text2_raw)

    # 标签1和标签2
    label1 = item.get('主要诊断编码', '')  # 单标签
    label2 = item.get('其他诊断编码', [])  # 多标签列表

    # 加入数据集
    dataset.append({
        'text1': text1,
        'text2': text2,
        'label1': label1,
        'label2': label2
    })

print(f"已处理数据条数：{len(dataset)}")

# 按9:1切分
train_data, val_data = train_test_split(dataset, test_size=0.1, random_state=42)

print(f"训练集大小：{len(train_data)}")
print(f"验证集大小：{len(val_data)}")

# 保存为JSON（如果需要）
with open(data_path + 'train_data/train_data.json', 'w', encoding='utf-8') as f:
    json.dump(train_data, f, ensure_ascii=False, indent=4)

with open(data_path + 'train_data/val_data.json', 'w', encoding='utf-8') as f:
    json.dump(val_data, f, ensure_ascii=False, indent=4)

print("训练集和验证集已保存完成！")


数据总量：800 条
已处理数据条数：800
训练集大小：720
验证集大小：80
训练集和验证集已保存完成！


### 统计切分后各个主要编码（或其他编码）的数量分布

In [ ]:
import json
from collections import Counter

# 读取JSON文件并统计诊断编码类别
def count_diagnosis_codes(json_file_path):
    # 存储所有诊断码的列表
    diagnosis_codes = []

    try:
        # 读取JSON文件
        with open(json_file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)

        # 遍历JSON数据
        for item in data:
            # 假设诊断编码存储在 'code' 字段中（需要根据实际JSON结构调整）
            if 'label2' in item:
                # 只取编码的前三个字符作为类别（如 I10, I20 等）
                code_category = item['label2']
                diagnosis_codes.append(code_category)

        # 使用Counter统计各类别的数量
        code_counts = Counter(diagnosis_codes)

        # 打印统计结果
        print("诊断编码类别统计结果：")
        for code, count in code_counts.items():
            print(f"{code}: {count} 次")

        return code_counts

    except FileNotFoundError:
        print(f"错误：找不到文件 {json_file_path}")
    except json.JSONDecodeError:
        print("错误：JSON文件格式不正确")
    except Exception as e:
        print(f"发生错误：{str(e)}")

# 示例用法
if __name__ == "__main__":
    # 假设JSON文件名为 'diagnosis.json'
    json_file = data_path + 'train_data/train_data.json'

    # 示例JSON数据结构可能如下：
    # [
    #   {"code": "I10.x00x032"},
    #   {"code": "I20.000"},
    #   {"code": "I20.800x007"},
    #   {"code": "I21.401"},
    #   {"code": "I50.900x018"}
    # ]

    count_diagnosis_codes(json_file)

诊断编码类别统计结果：
I49.900;I49.100x001;I49.300x002;I10.x00x027;J18.900;K76.000: 1 次
J98.414;I25.103;I50.900x010;I49.900: 1 次
I25.103;I48.x01;E87.600;I69.300x002: 1 次
I25.103;I10.x00x032;E11.900;I70.806;I70.203;R91.x00x003: 1 次
I10.x00x024;I70.203;I70.806;J98.414;E87.600: 1 次
I25.102: 1 次
E11.900: 5 次
Q24.501;I25.103;I49.900;I49.400x002;I49.300x002;R91.x00x003: 1 次
I10.x00x032;I48.x01;I49.900;I25.103;Z95.501: 1 次
I50.900x008;I49.900;I25.103: 1 次
I49.100x001;I70.806;I49.900;K76.807: 1 次
I10.x00x028;I69.300x002;R91.x00x003;K76.807: 1 次
I20.000;J98.414;I10.x00x032;I63.900: 1 次
I25.103;I38.x01;I10.x00x027;E11.900: 1 次
Q24.501;I10.x00x028;E11.900: 1 次
I50.900x008;I38.x01;I49.900;I48.x01;I25.103: 1 次
I20.000;I10.x00x031;I25.103: 1 次
I25.200;I25.103;I49.400x002;I49.300x002;I49.900;I10.x00x028;E11.900: 1 次
I10.x00x024;R91.x00x003;N28.101: 1 次
R91.x00x003;E04.101;K76.000: 1 次
I10.x00x024;I70.203;I70.806;I25.103: 1 次
I25.103;I10.x00x031;E11.900;I49.900;I49.300x001: 1 次
I25.103;I10.x00x023;I70.203;N28.10